In [31]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/battery_data.csv", index_col="datetime", parse_dates=True)
df.head()

,power_kW,pv_kW,autoconsommation_kW,injection_kW,soutirage_kW,soc,import_after_battery,export_after_battery
datetime,,,,,,,,
2023-01-01 00:00:00+00:00,53.882739,0.0,0.0,0.0,53.882739,0.0,53.882739,0.0
2023-01-01 01:00:00+00:00,38.719457,0.0,0.0,0.0,38.719457,0.0,38.719457,0.0
2023-01-01 02:00:00+00:00,53.568610,0.0,0.0,0.0,53.568610,0.0,53.568610,0.0
2023-01-01 03:00:00+00:00,68.411728,0.0,0.0,0.0,68.411728,0.0,68.411728,0.0
2023-01-01 04:00:00+00:00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [32]:
df["hour"] = df.index.hour
df["tarif"] = np.where((df["hour"] >= 22) | (df["hour"] < 6), 0.12, 0.18) # Tarif de 0.18€/kWh de 6h à 22h, sinon 0.12€/kWh
df["cout_import"] = df["import_after_battery"]* df["tarif"]
df["revenu_export"] = df["export_after_battery"] * 0.10 # Supposons un tarif de rachat de 0.10€/kWh pour l'export
df["cout_net"] = df["cout_import"] - df["revenu_export"]

df.head()
df.to_csv("../data/economy_data.csv", index = True)

In [33]:
cout_annuel = df["cout_net"].sum()
cout_sans_pv = (df["power_kW"] * df["tarif"]).sum()
cout_sans_batterie = (df["soutirage_kW"] * df["tarif"]).sum()
print(f"Coût annuel net : {cout_annuel:.2f} €")
print(f"Coût annuel sans batterie : {cout_sans_batterie:.2f} €")
print(f"Coût annuel sans PV : {cout_sans_pv:.2f} €")


Coût annuel net : 250887.79 €
Coût annuel sans batterie : 252811.63 €
Coût annuel sans PV : 375356.46 €


In [34]:
investissement_batterie = 50000 # Coût d'investissement pour la batterie
economie_annuelle = cout_sans_batterie - cout_annuel

roi = investissement_batterie / economie_annuelle if economie_annuelle > 0 else float('inf')
print(f"Économie annuelle : {economie_annuelle:.2f} €")
print(f"Retour sur investissement : {roi:.1f} ans")

Économie annuelle : 1923.84 €
Retour sur investissement : 26.0 ans
